# ECS713: Week 4 Lab Sheet

This lab sheet covers 
- partial application
- sections
- lambda expressions
- higher order functions
- apply, compose, curry and uncurry

## Learning Objectives

By the time you complete this sheet you should be able to
- understand and use partial applications
- use Haskell sections
- write Haskell lambda expressions
- understand what a higher order function is
- use and define higher order functions
- understand the functions apply, compose, curry and uncurry. 

## Turn off the annoying linter

Run the cell below to turn off the annoying linter, which suggests improvements to your code that aren't appropriate for these exercises. 

In [1]:
:opt no-lint

## Task 1. Partial Application

If a function takes several arguments, then we don't need to supply them all. If we don't, then the result is another 
function that needs the remaining arguments in order to produce a result. 

Example:
`take` has type `Int -> [a] -> [a]`. It expects two arguments, a number and a list, and produces a list as result. We can just supply the number. What we get is a function that takes a particular number of elements from the front of a list. 

In [2]:
takeThree = take 3

`takeThree` (or `take 3`) is a partial application of `take`. 

Test:

In [3]:
takeThree [1..10]
take 3 [1..10]
-- takeThree :: forall a. [a] -> [a]

[1,2,3]

[1,2,3]

Use partial application together with the functions, `map`, `filter`, `even`, `succ` to implement the following functions as partial applications: 

In [4]:
addOneAll = map succ

Test:

In [5]:
addOneAll [1..3] == [2..4]

True

In [6]:
:t addOneAll

addOneAll :: forall b. Enum b => [b] -> [b]

Define a function `filterEvens :: [Int] -> [Int]` that returns the even elements of a list. 

In [7]:
filterEvens = filter even

Test:

In [8]:
filterEvens [1..8] == [2,4,6,8]

True

In [9]:
:t filterEvens

filterEvens :: forall a. Integral a => [a] -> [a]

## Task 2. Sections

Some functions are infix (i.e. they sit between their arguments rather than in front of them). Infix functions can also be partially applied. The result is called a **section**. If the function is not commutative, then the left section is different from the right. 

Example:

In [10]:
map (2+) [1..10]
map (+2) [1..10]
map (^2) [1..10]
map (2^) [1..10]
map (2/) [1..10]
map (/2) [1..10]

[3,4,5,6,7,8,9,10,11,12]

[3,4,5,6,7,8,9,10,11,12]

[1,4,9,16,25,36,49,64,81,100]

[2,4,8,16,32,64,128,256,512,1024]

[2.0,1.0,0.6666666666666666,0.5,0.4,0.3333333333333333,0.2857142857142857,0.25,0.2222222222222222,0.2]

[0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0]

In [11]:
:t map

map :: forall a b. (a -> b) -> [a] -> [b]

Sections are very useful when the function is any of `==`, `/=`, `<` or `<=`.

In [12]:
map (=='a') "abracadabra"
map (/='a') "abracadabra"
map (<'b') "abracadabra"
map (<='b') "abracadabra"

[True,False,False,True,False,True,False,True,False,False,True]

[False,True,True,False,True,False,True,False,True,True,False]

[True,False,False,True,False,True,False,True,False,False,True]

[True,True,False,True,False,True,False,True,True,False,True]

Use `filter` and a section to remove all the spaces (' ') from the `String` below: 

In [13]:
testString = "This is a test string"
-- answer here
filterSpace = filter (not.(`elem` " "))
filterSpace testString

"Thisisateststring"

## Task 3. Lambda expressions

Sections and partial application do not solve all our problems. Here are two quick examples: 

The section `(2-)` is fine, but the section `(-2)` has problems (not surprisingly). 

Haskell has a function `mod :: Int -> Int -> Int`, `mod a b` gives the remainder on dividing `a` by `b`. Suppose we want a function that gives the remainder on division by 3. How do we get that? (One answer is that there is a bit of sleight of hand we can use to turn `mod` into an infix, and then use a section, but I am going to ignore that). 

(That sleight of hand: surround `mod` with backquotes, as in ``5 `mod` 3``. To go the other way, surround with parentheses: `(+) 2 3`).

It is easy to write little functions to do these, and then you can use them in local declarations (`let` or `where`). 

In [14]:
subTwo x = x-2
modThree a = mod a 3

But why introduce the new name? It's just clutter. Instead we can use a lambda expression. 

In [15]:
(\ x -> x-2) 5
(\a -> mod a 3) 5

3

2

This idea goes back to Church, as in the history lesson, and his lambda calculus (the \ is like a greek lambda). A lambda expression is a way of writing down a function without giving it a name. That separates the act of describing a function from the act of giving it a name. If we have a function definition: 

```foo x = <expression>```

then this is exactly equivalent to 

```foo = \x -> <expression>```

We have a lambda expression on the right, and we have given it a name. 

In Haskell lambda expressions can have multiple variables, and also use patterns as in:

In [16]:
plus = \ (x,y) -> x+y
plus (3,4)

7

This may not be the clearest way to write this, but just for fun, remove the definitions of `isTELLine` and `getTELFromLine` in the code below by replacing their use in `getTelFromVcard` with lambda expressions, so that the whole process is done with one line of code. 

In [17]:
isTELLine line = take 3 line == "TEL"
getTELFromLine line = tail $ dropWhile (/= ':') line
getTELFromVcard card = map getTELFromLine $ filter isTELLine $ lines card

In [18]:
:t lines

lines :: String -> [String]

In [19]:
:t getTELFromVcard

getTELFromVcard :: String -> [[Char]]

In [20]:
getTELFromVcard' :: String -> [[Char]]
getTELFromVcard'= \ card -> (map getTELFromLine $ filter isTELLine $ lines card)

Test on: 

In [21]:
cards = "BEGIN:VCARD\nVERSION:3.0\nPRODID:-//Apple Inc.//Mac OS X 10.9.5//EN\nN:Doe;Jane;;;\nFN:Jane Doe\nEMAIL;type=INTERNET;type=HOME;type=pref:jane.doe@notmail.com\nTEL;type=CELL;type=VOICE;type=pref:0123 456789\nTEL;type=WORK;type=VOICE:020 1234 5678\nADR;type=HOME;type=pref:;;1 Not Street;London;;A1 4BZ;\nitem1.URL;type=pref:jane.doe@me.net\nitem1.X-ABLabel:_$!<HomePage>!$_\nUID:8400c795-9dbc-487d-9d2a-de3311f9d075\nX-ABUID:8400C795-9DBC-487D-9D2A-DE3311F9D075:ABPerson\nEND:VCARD\nBEGIN:VCARD\nVERSION:3.0\nPRODID:-//Apple Inc.//Mac OS X 10.9.5//EN\nN:Doe;Jim Barnaby;;;\nFN:Jim Barnaby Doe\nEMAIL;type=INTERNET;type=WORK;type=pref:jim.doe@not.com\nTEL;type=CELL;type=VOICE;type=pref:02468 123456\nTEL;type=WORK;type=VOICE:020 2345 6789\nUID:7476fd69-9bf1-416e-b093-05ddc23c385f\nX-ABUID:7476FD69-9BF1-416E-B093-05DDC23C385F:ABPerson\nEND:VCARD\nBEGIN:VCARD\nVERSION:3.0\nPRODID:-//Apple Inc.//Mac OS X 10.9.5//EN\nN:Doe;John;;;\nFN:John Doe\nORG:Queen Mary;\nEMAIL;type=INTERNET;type=HOME;type=pref:JohnDoe@nogmail.com\nTEL;type=CELL;type=VOICE;type=pref:0751 234567\nTEL;type=HOME;type=VOICE:020 7123 4567\nADR;type=HOME;type=pref:;;42 Nowhere St;London;;E1 0XX;\nUID:8bb3d68ec846f1\nX-ABUID:608AE44E-BF05-4713-A748-AAE3CAB64858:ABPerson\nEND:VCARD\n"
getTELFromVcard' cards

["0123 456789","020 1234 5678","02468 123456","020 2345 6789","0751 234567","020 7123 4567"]

## Task 4. Recognising Higher Order Functions

**Simplified Definition:** A function is higher-order if one of its arguments is also a function. 

This means that you can tell whether a function is higher order or not by looking at its type, and checking the types of this arguments. If you can't see a function anywhere there, then it is first order. If you can, then it is higher order. The reason this definition is oversimplified is that the function might not be quite in plain sight. It might, for instance, be part of an algebraic datatype. 

Example:

In [22]:
:t drop

drop :: forall a. Int -> [a] -> [a]

In [23]:
:t dropWhile

dropWhile :: forall a. (a -> Bool) -> [a] -> [a]

The arguments to `drop` have types `Int` and `[a]`. It is first order. 

The arguments to `dropWhile` have types `a -> Bool` and `[a]`. Since `a -> Bool` is a function type, it is higher order. 

In [24]:
data PackedFunction a = Pack (a -> Bool)
dropWhile' (Pack f) = dropWhile f
:t dropWhile'

dropWhile' :: forall a. PackedFunction a -> [a] -> [a]

This is a blatant attempt at cheating the definition. I've introduced a datatype whose only purpose is to put a wrapper round a function. When I look at the type of `dropWhile'`, there is no function type visible, just `PackedFunction a` and `[a]`. It is still higher order. 

Here is a list of functions from the Haskell prelude (so standard functions), together with their types (in some cases I've simplified them a bit). Identify the ones that are higher order and give your evidence by providing the type of the function that is an argument:

## Task 5. Using and Defining Higher Order Functions

If we have a higher order function, then there are lots of ways we can supply the function it expects as argument:
- as a named function
- as a partial application
- as a section
- as a lambda expression

We've already done this. 

Find examples in these exercises where we have used higher order functions like `map` and `filter`. 

If we want to define a higher order function, then we can use the same methods as before. For example we can define functions on lists by recursion:

In [25]:
map' :: (a -> b) -> [a] -> [b]
map' f [] = []
map' f (a:as) = (f a) : (map f as)

In [26]:
map' (+1) [1..4]

[2,3,4,5]

Or:

In [27]:
takeWhile' :: (a -> Bool) -> [a] -> [a]
takeWhile' p [] = []
takeWhile' p (a:as) = if (p a) then a : (takeWhile' p as) else []

In [28]:
takeWhile' (<5) [1..10]

[1,2,3,4]

Give your own similar definitions of `filter'` and `dropWhile'`.

In [29]:
-- code here
filter' :: (a -> Bool) -> [a] -> [a]
filter' f [] = []
filter' f (a:as) = if (f a) then a : (filter' f as) else filter' f as

dropWhile' :: (a -> Bool) -> [a] -> [a]
dropWhile' f [] = []
dropWhile' f (a:as) = if (f a) then (dropWhile' f as) else as

In [30]:
-- tests here
filter' even [1..4]
dropWhile' (<3) [1..5]

[2,4]

[4,5]

In [31]:
:t dropWhile'

dropWhile' :: forall a. (a -> Bool) -> [a] -> [a]

## Task 6: Higher order functions apply and compose

Haskell is a higher order language and is designed so that functions are first class objects. They are treated in exactly the same way as first-order and other data. As a consequence we have functions that manipulate other functions to produce still more functions. 

At a very basic level we might have a function that swaps the order of the parameters of a binary function: 

In [32]:
swap2 f a b = f b a
:t swap2

swap2 :: forall t1 t2 t3. (t1 -> t2 -> t3) -> t2 -> t1 -> t3

So if we look at `swap2 (-)` we should get the reverse of `-`:

In [33]:
m = swap2 (-)
m 4 3

-1

Define a function `diag` so that `diag f a` returns `f a a`.

In [34]:
diag :: (a -> a -> a) -> a -> a
diag f a = f a a

In [35]:
add = diag (+) 
multi = diag (*)

In [36]:
add 3

6

In [37]:
multi 3

9

Describe the functions `diag (+)` and `diag (*)`:

Explanation:

So that was the warm up. Now we come to the functions actually used in practice. First `curry` and `uncurry`. If you want to give two arguments to a function, you can either give them as two separate arguments, or you can give them as a single pair. `curry` and its inverse `uncurry` enable you to swap between these choices, as you can see from their types: 

In [38]:
:t curry
:t uncurry

curry :: forall a b c. ((a, b) -> c) -> a -> b -> c

uncurry :: forall a b c. (a -> b -> c) -> (a, b) -> c

For example: 

In [39]:
add = uncurry (+)

Usage:

In [47]:
add (3,4)
(+) 3 4 

7

7

We'll see examples later. 

**Function composition:** When we construct a data pipeline we are composing functions. Haskell has a higher order function that lets you do that.  It is written `.` so is almost invisible, and it is infix: 

In [41]:
:t (.)

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

The code from lecture 1 was: 

In [42]:
isTELLine line = take 3 line == "TEL"
getTELFromLine line = tail $ dropWhile (/= ':') line
getTELFromVcard card = map getTELFromLine $ filter isTELLine $ lines card

In [43]:
:t isTELLine
:t dropWhile (/= ':')
:t map getTELFromLine
:t filter isTELLine

isTELLine :: [Char] -> Bool

dropWhile (/= ':') :: [Char] -> [Char]

map getTELFromLine :: [[Char]] -> [[Char]]

filter isTELLine :: [[Char]] -> [[Char]]

We can rewrite these using composition: 

In [44]:
getTELFromLine' = tail . (dropWhile (/= ':'))
getTELFromVcard' = (map getTELFromLine') . (filter isTELLine) . lines

If you have the annoying linter on it will complain about a lot of redundant brackets here. 

You use compose when you have two functions you need to put together, and the result of one is the argument of the other. 

**Function application:** Look at the type of `$`:

In [45]:
:type ($)

($) :: forall a b. (a -> b) -> a -> b

`$` takes two arguments, one of type `a -> b` (a function) and the other of type `a`. It returns something of type `b`, specifically the result of applying its first argument to its second. `$` is just function application written as a visible higher order function. (Usually function application is entirely invisible, we just write the function next to the argument we want to apply it to.)

You use function application when you have a function and a value, and the type of the argument of the function is the type of the value. 

Types `a -> b` and `b -> c`: use composition (`.`)
Types `b -> c` and `b`: use application (`$`)

We will test this in quizzes. 

In [50]:
info ($)


: 